#### Les imports

In [1]:
from tkinter import *
from random import randrange
from math import copysign

# LES FONCTIONS 

In [2]:
def changer_couleur (i,colour):
    """change de couleur le rectangle à l'indice i dans la liste L """
    canvas.itemconfigure( L[i][0],fill=colour)
    L[i][3]=colour

#### Création du terrain 

In [3]:
def creation_terrain ():
    """ Crée le terrain de base avec des rectangles (20*20) seulement vert.
    Les dimensions des rectangles nous obligent à avoir un Canvas de hauteur et de longueur divisible par 20
    Nous mettons toutes les informations relatifs à chaque rectangle dans la liste L"""
    for j in range (0,hauteur,20):
        for i in range (0,longueur,20):
            rectangle = canvas.create_rectangle(i,j,i+20,j+20,fill="green")
            L.append ([rectangle,i,j,"green",0]) # //0 -> le rectangle//1 -> sa coordonéee x// 2 -> y //3 -> sa couleur//4-> compteur cendre//

In [4]:
def vide():
    """ crée 100 cases vides de couleur blanche aléatoirement dans le terrain"""
    k=0
    while (k<100): 
        i=randrange(0,len(L))
        k=k+1
        changer_couleur(i,"white")

In [5]:
def initialisation_feu ():
    """ fonction associée à un bouton qui rend UNE case rouge aléatoirement"""
    i= randrange( nb_cases)
    while (L[i][3]!="green"):
        i= randrange( nb_cases)
    changer_couleur (i,"red")    
    

In [6]:
def case_eau(n,i):
    """ fonction qui rend bleu le rectangle d'indice i et les n rectangles au dessus
    On se servira de cette fonction pour créer des rivières aléatoires """
    changer_couleur(i,"blue")
    signe=int( copysign(1,n) )
    
    k=0
    while (k!=n):
        k+=signe
        if ( (signe>0 and i+30 <=899) or (signe < 0 and i-30 >=0)):
            i=i + 30*signe
            changer_couleur(i,"blue")
    

In [7]:
def riviere_aleatoire():
    """ fonction associée à un bouton qui joint deux points aléatoires situées sur deux côté différents du Canvas
    avec une rivière d'épaisseur aléatoire"""
    L1=["droite","gauche","haut","bas"]
    depart=L1[randrange(2)] # depart vaut aléatoirement droite ou gauche
    direction=L1[randrange(2,4)] # direction vaut aléatoirement haut ou bas
    if (depart=="gauche"):
        depart= randrange (nb_colonne)*30 # il s'agit de toutes les cases de la première colonne 
        dx=+1
    if (depart=="droite"):
        depart =randrange(1,nb_colonne+1)*30 -1 #il s'agit de toutes les cases de la dernière colonne 
        dx=-1
    e= randrange(1,4) # L'épaisseur aléatoire de la rivière
    if (direction =="haut"):
        e=-e
        while (depart-nb_colonne>0): # tant qu'on atteind pas la première ligne
            case_eau(e,depart) # on rend les e cases au dessus de depart et lui même bleues
            depart= depart - nb_colonne+dx # on déplace départ en diagonale 
    
    if (direction =="bas"):
        while (depart+nb_colonne<900): # tant qu'on atteind pas la dernière ligne 
            case_eau(e,depart) # on rend les e cases au dessus de lui bleues
            depart= depart + nb_colonne+dx # on déplace départ en diagonale 

#### L'animation de propagation du feu 

In [8]:
def case_adjacente(i):
    """ Prend en paramètre l'indice d'une case en feu, retourne une liste des cases vertes adjacentes à cette case en feu
    qui ont passé le test de propabilité. Retourne une liste d'indice de case à enflammer"""
    M=[]
    #case du haut
    if (L[i][2]!=0 and L[i-nb_colonne][3]=="green" and test_case("nord")):
        M.append(i-nb_colonne)
    #case du bas
    if (L[i][2]!=hauteur-20 and L[i+nb_colonne][3]=="green" and test_case("sud")):
        M.append(i+nb_colonne)
    # case de gauche 
    if (L[i][1]!=0 and L[i-1][3]=="green" and test_case ("ouest")):
        M.append(i-1)
    #case de droite
    if (L[i][1]!=longueur-20 and L[i+1][3]=="green" and test_case("est")):
        M.append(i+1)
    return M

In [9]:
def propagation_feu ():
    """ fonction qui se rappelle elle-même toutes les 100 ms. Elle envoie à la fonction case_adjacente toutes les cases 
    enflammées. On stocke dans la liste M toutes les indices des cases renvoyées par case_adjacente. Ce seront les cases
    à enflammées. On les enflamme et toutes les cases rouge dont le compteur_cendre devient égal à tour_cendre deviennent grises.
    Ce sont des cendres qui ne seront plus imflammables """ 
    FEU=[]
    CENDRE=[]
    for i in range ( 0,len(L)):
        if (L[i][3]=="red"):
            FEU.extend(case_adjacente(i))
            L[i][4]+=1
            if (L[i][4] == tour_cendre):
                CENDRE.append(i)
    for j in FEU:
        changer_couleur(j,"red")
    for k in CENDRE:
        changer_couleur (k,"grey")
    fen.after (100,propagation_feu)    
        

#### Le test de probabilité 

In [10]:
def direction_vent(event):
    """ fonction qui assigne à la variable direction la chaine de caractère rentrée par l'utilisateur.
    La direction du vent, pourra être nord,est,ouest ou sud en miniscule"""
    global direction 
    direction = entre_direction.get()
    text1.configure(text="Direction du vent: "+direction)

In [11]:
def puissance_vent (event):
    """ fonction qui assigne à la variable puissance l'entier rentré par l'utilisateur.
    La puissance du vent est en m/s, pour rester réaliste sa valeur pourra
    aller jusqu'à 25 m/s (90 km/h)"""
    global puissance
    puissance = int (entre_puissance.get())
    text2.configure (text="Puissance du vent: "+str(puissance)+" m/s")

In [12]:
def reserve_eau(event):
    """ fonction qui assigne à la variable reserve l'entier entré par l'utilisateur
    La réserve en eau maximale du sol est de 150 mL, elle baisse
    exponentiellement avec la fin de la période des pluies.
    Si la valeur rentrée est entre 101 et 150, il s'agit d'une sécheresse faible.
    Si la valeur rentrée est entre 51 et 100, il s'agit d'une sécheresse moyenne.
    Si la valeur rentrée est entre 31 et 50, il s'agit d'une sécheresse forte.
    Si la valeur rentrée est entre 0 et 30, il s'agit d'une sécheresse très forte."""
    global reserve
    reserve = int(entre_reserve.get())/6
    text3.configure(text="Réserve en eau du sol: "+str(reserve*6)+" mL")
    # On divise cette valeur par 6 pour que les deux paramètres aient à peu près la même influence ( 150/6==25)

Après des recherches sur les paramètres provoquants un incendie, on remarque que
les deux seuls paramètres réellement influents sont la vitesse moyenne du vent 
et la reserve en eau du sol. Plus le vent est fort, plus le risque d'incendie est 
élevé et plus la réserve en eau du sol est faible plus le risque d'incendie est élevé.
Les autres paramètres auxquels on peut penser comme l'humidité de l'air, la température,
ou encore le type de végétation sont pas ou trop peu influents pour les prendre en
considération dans notre simulation.
L'influence de ces deux paramètres retenus est à peu près équivalente.

In [13]:
def test(p):
    """ fonction qui a p% de chance de renvoyer True et (100-p)% de chance de renvoyer False"""
    a = randrange(1,101)
    if ( a> p):
        return False
    else:
        return True 

In [14]:
def test_case(x):
    """ fonction qui prend en paramètre la direction d'une case adjacente à une case rouge dans la liste L
    Retourne retourne True si la case passe le test, False sinon.
    La probabilité p de passer le test diminue fortement avec l'augmentation de la réserve en eau du sol.
    La probabilité p de passer le test augmente fortement si la direction de la case adjacente est dans le sens du vent.
    La probabilité p de passer le test diminue si la direction de la case adjacente est dans le oppposé au vent.
    La probabilité p de passer le test augmente faiblement si la direction de la case adjacente est perpendiculaire au vent.
    """
    global cp1,cp2,cp3 # servent seulement à print UNE FOIS p pour chaque cas.
    a = x[0] + direction[0] 
    M=["oe","eo","ns","sn"]
    p1=p - reserve * 3
    if (x == direction):# cas où la direction de la case est dans le sens du vent
        p1 = p1 + puissance*3
        if (cp1==0):
            print("dans le sens du vent p= ",p1)
            cp1+=1
            
        
    elif (a in M):#cas où la direction de la case est dans le sens opposé au vent
        p1=p1-puissance
        if (cp2==0):
            print ("dans le sens oppose au vent p= ",p1)
            cp2+=1
    else : # cas où la direction de la case est perpendiculaire au sens du vent
        p1= p1+puissance
        if (cp3==0):
            cp3+=1
            print ("dans le sens perpendiculaire au vent p= ",p1)
         
    return test(p1) # on retourne le test

# PROGRAMME PRINCIPAL

#### Les variables

In [15]:
L= []
hauteur =600 # hauteur et longueur doivent être des multiples de 20
longueur=600
tour_cendre=1 # Le nombre de tour avant qu'une case enflammée ne soit plus que cendre.
p=100
nb_ligne=int(longueur/20)
nb_colonne=int(hauteur/20)
nb_cases = nb_colonne * nb_ligne 
nb_cases == len(L)
puissance=0
reserve=0
direction ="aucune"

cp1=0
cp2=0
cp3=0


#### Création des widgets

In [16]:
fen = Tk()
entre_puissance=Entry (fen)
entre_puissance.bind ("<Return>",puissance_vent)
entre_direction=Entry (fen)
entre_direction.bind ("<Return>",direction_vent)
entre_reserve=Entry(fen)
entre_reserve.bind("<Return>",reserve_eau)
text1 = Label(text="Direction du vent: "+direction)
text2 = Label (text="Puissance du vent: "+str(puissance)+" m/s")
text3= Label (text="Réserve en eau du sol: "+str(reserve)+" mL")
canvas = Canvas (fen, height =600,width=600,bg="white")

#### Le placement des widgets

In [17]:
Label(text="Reserve d'eau du sol:   entre 0 et 150 mL").grid(row=5,column=0,sticky=E)
entre_reserve.grid(row=5,column=1,sticky=W)
Label (text="Puissance du vent:   entre 0 et 25 m/s").grid(row=6,column=0,sticky=E)
entre_puissance.grid(row=6,column=1,sticky=W)
Label (text="Direction du vent :   ouest,est,sud ou nord").grid(row=7,column=0,sticky=E)
entre_direction.grid(row=7,column=1,sticky=W)
Button (text="initialisation feu", command=initialisation_feu).grid(row=6,column=2)
Button (text="riviere aleatoire",command=riviere_aleatoire).grid(row=5,column=2)
Button (text="propagation feu",command=propagation_feu).grid(row=7,column=2)
text1.grid (row=0,column=0,sticky=W)
text2.grid (row=0,column=1)
text3.grid (row=0,column=2,sticky=E)
canvas.grid (row=1,column=0,rowspan=3,columnspan=3)

In [18]:
creation_terrain()
vide()
fen.mainloop()

dans le sens oppose au vent p=  37.5
dans le sens du vent p=  137.5
dans le sens perpendiculaire au vent p=  87.5
